# Fog and Storm Detection - Climate Risk and Disaster Management Project
This project explores a dataset related to fog and storm events, focusing on climate risk and disaster management.

In [ ]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('fog_storm_dataset.csv')
df.head()

In [ ]:
# Dataset info
df.info()

In [ ]:
# Statistical summary
df.describe()

In [ ]:
# Missing values check
df.isnull().sum()